In [ ]:
import pandas as pd
from collections import Counter
from langdetect import detect
import langdetect
import numpy as np
import importlib

current_dir = os.getcwd()
%cd ..
import textmining.text_miner 
import textmining.topic_modeler as tm

importlib.reload(textmining.text_miner)
importlib.reload(textmining.topic_modeler)
os.chdir(current_dir)

%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

# Mallet LDA train

In [ ]:
data = pd.read_csv('../../data/processed/final_repo_english_whatwhy.csv', index_col='file_id')

stop_words_extension = ['abstr_hyperlink', 'https', 'abstr_number', 'abstr_image',
                        'abstr_mailto', 'abstr_code_section', 'abstr_image', 'abstr_number', 'http', 'www', 'please']

mallet_lda_modeler = tm.topic_modeler(data['content_clean'].values.tolist(),
                               stopwords_extension= stop_words_extension)


In [ ]:
len(data)

## Plotting cohenrence for each K topics

In [ ]:
start= 40#125## 85 # 45
step=1
limit=146 # not included


In [ ]:
_, coherence_values = mallet_lda_modeler.compute_coherence_values(limit, start=start, step=step, model_type='mallet')


In [ ]:
## Saving results 
n = mallet_lda_modeler.num_topics
print('best model has {} topics'.format(n))

coherence_vals_df =  pd.DataFrame({'Number of Topics':range(start, limit, step), 'Coherence Value':coherence_values} )
coherence_vals_df.to_csv('../../data/feature_extraction/lda/ldamallet_coherence_values_{}_{}.csv'.format(start, (limit-1)))

In [ ]:
def save_model_info(modeler):
    n = modeler.num_topics
    print('get_doc_dominant_topic')
    nmf_readable_document_topics_df =  modeler.get_doc_dominant_topic(save_path= "../../data/feature_extraction/lda/ldamallet_readable_document_topics_{}.csv".format(n))
    
    print('get_topics_terms')
    nmf_topic_words_df = modeler.get_topics_terms(save_path= "../../data/feature_extraction/lda/ldamallet_topic_words_{}.csv".format(n))
    
    print('get_doc_topic_matrix')
    nmf_document_topics_df =  modeler.get_doc_topic_matrix(save_path= "../../data/feature_extraction/lda/ldamallet_document_topics_{}.csv".format(n))
    
    return nmf_readable_document_topics_df, nmf_topic_words_df, nmf_document_topics_df

In [ ]:
lda_readable_document_topics_df, lda_topic_words_df, lda_document_topics_df = save_model_info(mallet_lda_modeler)

## Topics Exploration

#### Topics Keywords

In [ ]:
topics_with_perc_df = pd.DataFrame(mallet_lda_modeler.model.print_topics(num_topics=40, num_words=10), 
                                   columns=['Topic', 'Keywords'])
#topics_with_perc_df.reset_index().to_csv('../../data/feature_extraction/lda_topic_words_45.csv')
mallet_lda_modeler.model.show_topics(num_topics=45, num_words=10, formatted=False)

#### Docs dominant Topics

#### Topic Distribution - See below (Wordlcloud for topic top words)

In [ ]:
topic_dstr = mallet_lda_modeler.get_topic_distr()

In [ ]:
topic_dstr['Dominant_Topic'].value_counts().plot(kind='bar')

In [ ]:
topic_dstr.to_csv('../../data/feature_extraction/lda/ldamallet_topics_distribution_{}.csv'.format(n))

#### Word Cloud for each Topic

In [ ]:
## word cloud
from nltk.corpus import stopwords
from matplotlib import pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import matplotlib.colors as mcolors
def show_word_cloud(model, topics, limit=10, stopwords_extension=[], topic_index=range(1,11)):
    # 1. Wordcloud of Top N words in each topic


    cols = [color for name, color in mcolors.TABLEAU_COLORS.items()]  # more colors: 'mcolors.XKCD_COLORS'

    
    stop_words  = stopwords.words('english')
    stop_words.extend(stopwords_extension)
        
    cloud = WordCloud(stopwords=stop_words,
                    background_color='white',
                    width=2500,
                    height=1800,
                    max_words=10,
                    colormap='tab10',
                    color_func=lambda *args, **kwargs: cols[i],
                    prefer_horizontal=1.0)

    
    fig, axes = plt.subplots( int(limit/2), 2, figsize=(10,10), sharex='all', sharey='all')


    for i, ax in enumerate(axes.flatten()):
        fig.add_subplot(ax)
        topic_words = dict(topics[i][1])
        cloud.generate_from_frequencies(topic_words, max_font_size=300)
        plt.gca().imshow(cloud)
        plt.gca().set_title('Topic ' + str(topic_index[i]), fontdict=dict(size=16))
        plt.gca().axis('off')
        if i> limit:
            break


    plt.subplots_adjust(wspace=0, hspace=0)
    plt.axis('off')
    plt.margins(x=0, y=0)
    plt.tight_layout()
    plt.show()

In [ ]:
topics = LDA_MODEL.show_topics(num_topics=35, num_words=10, formatted=False)


In [ ]:
show_word_cloud(LDA_MODEL, topics[:10],  limit=10, stopwords_extension=stop, topic_index=range(0,10))

In [ ]:
show_word_cloud(LDA_MODEL, topics[10:20],  limit=10, stopwords_extension=stop, topic_index=range(10,20))

In [ ]:
show_word_cloud(LDA_MODEL, topics[20:30],  limit=10, stopwords_extension=stop, topic_index=range(20, 30))

In [ ]:
show_word_cloud(LDA_MODEL, topics[30:35],  limit=5, stopwords_extension=stop,  topic_index=range(20, 35))